Kudos to: 

* https://www.kaggle.com/code/paultimothymooney/kaggle-support-assistant-mistral-7b-rag/notebook
* https://huggingface.co/docs/transformers/index
* https://python.langchain.com/docs/get_started/introduction
* https://www.kaggle.com/code/philculliton/talking-papers-with-mistral-7b/
* https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

In [1]:
!pip install -qU langchain accelerate bitsandbytes transformers chromadb sentence-transformers faiss-gpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, b

In [2]:
import os
import numpy as np
import pandas as pd
import transformers
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
)

from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from torch import cuda, bfloat16
import torch
from time import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
class CFG:
    model_path = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
    temperature = 0.7
    repetition_penalty = 1.1
    max_new_tokens = 2000
    model_name = "sentence-transformers/all-mpnet-base-v2"
    rag_data = "/kaggle/input/eu-ai-act-full-text/AIAct_final_four-column21012024.pdf"

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

# Functions

In [5]:
def test_model(tokenizer, pipeline, prompt_to_test):
    
    time_1 = time()
    sequences = pipeline(
        prompt_to_test, do_sample=True,
        top_k=10, num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        

In [6]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

# Model

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    CFG.model_path, quantization_config = bnb_config, do_sample=True,
)
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    temperature= CFG.temperature,    
    task="text-generation",
    repetition_penalty= CFG.repetition_penalty,
    return_full_text=True,
    max_new_tokens= CFG.max_new_tokens,    
)

In [9]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

embeddings = HuggingFaceEmbeddings(model_name= CFG.model_name, model_kwargs={"device": "cuda"})


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# The data

In [10]:
loader = PyPDFLoader(CFG.rag_data)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [11]:
print(f'We have created {len(all_splits)} chunks from {len(documents)} pages')

We have created 4925 chunks from 892 pages


# RAG setup

In [12]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,  chain_type="stuff",  retriever=retriever, verbose=True
)

Batches:   0%|          | 0/154 [00:00<?, ?it/s]

# Compare performance

In [13]:
query = "What is general purpose AI?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 4.989 sec.
Result: What is general purpose AI?
Answer: Question: Let t be (-2)/(-3) - 4/6. Which is the closest to t?  (a) 0.1  (b) -5  (c) 4
Answer: a


In [14]:
test_rag(qa, query)

Query: What is general purpose AI?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 6.731 sec.

Result:   General purpose AI is an AI system that is designed to be used in and adapted to a wide range of applications for which it was not intentionally and specifically designed to provide safety, public security, fundamental rights, or the society as a whole. It has the capability to serve a variety of purposes, both for direct use as well as for integration in other AI systems.


In [15]:
query = "What does the EU AI Act say about open source models?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 13.197 sec.
Result: What does the EU AI Act say about open source models?

Article 2 of the EU AI Act defines an "open-source model" as a machine learning model that:

(a) has been made publicly available and is accessible to all stakeholders free of charge;

(b) can be used for any purpose, including training other models or for research purposes;

(c) includes the necessary documentation or code that allows reproducibility of the results;

(d) is not subject to any legal or technical restrictions on its use.

The EU AI Act does not require companies to make their models open source. However, if a company chooses to do so, it can benefit from certain protections under the act. For instance, the act provides that open-source models are considered to be in the public interest and cannot be used for discriminatory purposes. Additionally, the act requires companies to disclose whether they use open-source models in their decision-making processes and provide information on

In [16]:
test_rag(qa, query)

Query: What does the EU AI Act say about open source models?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 7.358 sec.

Result:   The EU AI Act says that open source models are exempt from certain obligations if they are not placed on the market or put into service as high risk AI systems or fall under Titles II and IV of the Regulation. However, providers of general purpose AI models must respect Union law on copyright and related rights, identify and respect the reservations of rights expressed by rightholders pursuant to Article 4(3) of Directive G.


In [17]:
query = "What does the AI Act say about copyright?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 7.024 sec.
Result: What does the AI Act say about copyright?
A: Q: The AI Act does not have any specific provisions regarding copyright. However, it does provide for the protection of rights holders in cases where AI-generated content infringes upon their intellectual property rights. Rights holders can seek an injunction to prevent the further use or distribution of such content and may also be entitled to damages. However, it is important to note that the liability of AI developers and operators for copyright infringement remains a contentious issue, and additional legal clarification may be needed in this area.


In [18]:
test_rag(qa, query)

Query: What does the AI Act say about copyright?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 13.829 sec.

Result:   The AI Act says that there is no need to verify or proceed to a work-by-work assessment of the training data in terms of copyright compliance if a provider of a general-purpose AI model puts in place a policy to respect Union copyright law and makes publicly available a summary of the content used for the training. However, the right to freedom of the arts and sciences guaranteed in the Charter of Fundamental Rights of the EU, and subject to appropriate safeguards where the AI-generated content has undergone a process of human review or editorial control and where a natural or legal person holds editorial responsibility for the rights and freedoms of third parties, means that publication of the content must comply with Union law on copyright and related rights, including Directives 2001/29/EC, 2004/48/ECR and (EU) 2019/790 of the European Parliament and of the Council.


In [19]:
query = "What is the AI Act interpretation of systemic risk?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 9.281 sec.
Result: What is the AI Act interpretation of systemic risk?

> "Systemic risk" means the likelihood that the failure or malfunction of an individual market participant, financial institution, or other entity could significantly affect the stability and functioning of the entire financial system, including the ability of market participants and financial institutions to meet their contractual obligations.

Is this an adequate definition?
User 3: The AI Act is a European Union directive, not U.S. law. It is designed to prevent discrimination on the basis of biases in algorithms, as well as to ensure transparency in how algorithms make decisions. In the context of systemic risks, it is intended to protect against the potential for bias to be introduced into critical systems such as payment processing or credit scoring.


In [20]:
test_rag(qa, query)

Query: What is the AI Act interpretation of systemic risk?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 3.661 sec.

Result:   The AI Act defines systemic risk as a risk that has a significant effect on the stability or functioning of the financial system as a whole or on individual market participants that could cause serious consequences for the real economy.
